Gabriel Avinaz\
Week 7\
4/26/23

## Assignment 7.1
### Part a

In [52]:
import os
import shutil
import pandas as pd

In [53]:
path = "data"
if not os.path.exists(path):
    os.makedirs(path)
    shutil.copy("../assignment03/results/routes.parquet", path)

In [54]:
routes_df = pd.read_parquet("data/routes.parquet", engine='pyarrow')
routes_df = pd.json_normalize(routes_df["Flight Info"])
routes_df.sample(10)

,codeshare,equipment,airline.active,airline.airline_id,airline.alias,airline.callsign,airline.country,airline.iata,airline.icao,airline.name,...,src_airport.icao,src_airport.latitude,src_airport.longitude,src_airport.name,src_airport.source,src_airport.timezone,src_airport.type,src_airport.tz_id,dst_airport,src_airport
54155,False,"[736, 32A]",True,4870,TAP Air Portugal,TUNAIR,Tunisia,TU,TAR,Tunisair,...,DTMB,35.758099,10.754700,Monastir Habib Bourguiba International Airport,OurAirports,1.0,airport,Africa/Tunis,NaN,NaN
8482,False,"[E90, 319, 320, 763, E75]",True,330,ANA All Nippon Airways,AIR CANADA,Canada,AC,ACA,Air Canada,...,CYYZ,43.677200,-79.630600,Lester B. Pearson International Airport,OurAirports,-5.0,airport,America/Toronto,NaN,NaN
248,False,[AT7],False,1729,SN Brussels Airlines,CHANGAN,China,2Z,CGN,Changan Airlines,...,SBRP,-21.136389,-47.776669,Leite Lopes Airport,OurAirports,-3.0,airport,America/Sao_Paulo,NaN,NaN
21840,False,[320],True,2009,CSA Czech Airlines,DELTA,United States,DL,DAL,Delta Air Lines,...,KSLC,40.788399,-111.977997,Salt Lake City International Airport,OurAirports,-7.0,airport,America/Denver,NaN,NaN
23028,False,[320],True,837,Pulkovo Aviation Enterprise,SHAMROCK,Ireland,EI,EIN,Aer Lingus,...,EICK,51.841301,-8.491110,Cork Airport,OurAirports,0.0,airport,Europe/Dublin,NaN,NaN
57589,True,[CR7],True,5209,TWA,UNITED,United States,UA,UAL,United Airlines,...,KSAT,29.533701,-98.469803,San Antonio International Airport,OurAirports,-6.0,airport,America/Chicago,NaN,NaN
5838,True,[320],True,24,\N,AMERICAN,United States,AA,AAL,American Airlines,...,EGLL,51.470600,-0.461941,London Heathrow Airport,OurAirports,0.0,airport,Europe/London,NaN,NaN
48395,False,[320],True,4304,Qantas Airways,AIR AZORES,Portugal,S4,RZO,SATA International,...,LPPT,38.781300,-9.135920,Humberto Delgado Airport (Lisbon Portela Airport),OurAirports,0.0,airport,Europe/Lisbon,NaN,NaN
50932,False,[319],True,130,\N,AEROFLOT,Russia,SU,AFL,Aeroflot Russian Airlines,...,ULLI,59.800301,30.262501,Pulkovo Airport,OurAirports,3.0,airport,Europe/Moscow,NaN,NaN
61585,False,[321],True,5309,TWA,VIET NAM AIRLINES,Vietnam,VN,HVN,Vietnam Airlines,...,VVPC,13.955000,109.042000,Phu Cat Airport,OurAirports,7.0,airport,Asia/Saigon,NaN,NaN


In [55]:
def key_gen(row):
    return str(row['src_airport.iata']) + str(row['dst_airport.iata']) + str(row['airline.iata'])

In [56]:
routes_df["key"] = routes_df.apply(key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key
20311,BJX,ATL,DL,BJXATLDL
50111,PMI,CPH,SK,PMICPHSK
50113,PMI,OSL,SK,PMIOSLSK
18568,HGH,XIY,CZ,HGHXIYCZ
20640,DTW,LHR,DL,DTWLHRDL
1543,PMI,TXL,4U,PMITXL4U
16058,CAN,GYS,CA,CANGYSCA
62530,ENU,ABV,W3,ENUABVW3
55284,OPO,CDG,U2,OPOCDGU2
34573,HND,CTS,JL,HNDCTSJL


In [57]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

def kv_key_gen(row):
    for tuple in partitions:
        first_char = row["key"][:1]
        if first_char >= tuple[0] and first_char <= tuple[1]:
            if tuple[0] == tuple[1]:
                return tuple[0]
            else:
                return tuple[0] + '-' + tuple[1]

In [58]:
routes_df["kv_key"] = routes_df.apply(kv_key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'kv_key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,kv_key
9858,ORY,PTP,AF,ORYPTPAF,O-P
26248,BRU,ALC,FR,BRUALCFR,B
18884,NNG,HGH,CZ,NNGHGHCZ,N
32812,ORD,DUS,IB,ORDDUSIB,O-P
59418,JFK,BOG,US,JFKBOGUS,I-J
10927,CUL,MTY,AM,CULMTYAM,C-D
35091,BDO,JOG,JT,BDOJOGJT,B
19447,YCU,CKG,CZ,YCUCKGCZ,Y-Z
48872,JNB,MRU,SA,JNBMRUSA,I-J
17668,PTY,CUN,CM,PTYCUNCM,O-P


In [59]:
routes_df.to_parquet('results/kv/', partition_cols=['kv_key'])

### Part b

In [61]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [62]:
routes_df["hashed"] = routes_df.apply(lambda x: hash_key(x["key"]), axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'hashed']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,hashed
60576,BIO,IBZ,UX,BIOIBZUX,898a50f70d6b192ed05577a8e68762242ba2865de33a80...
41484,CTU,XIY,MU,CTUXIYMU,bbc4f2f62aa6085d4e2ca76eb7de857679078948b3be3e...
6476,PHL,PBI,AA,PHLPBIAA,7cdae514c2c5506b2e86ce62bb0ccfe9a7e89db2f39ca8...
42550,XMN,NGB,MU,XMNNGBMU,3858e2fab8da655823a1657d62d420416a7de4d680fcce...
9371,CDG,NRT,AF,CDGNRTAF,efbadb561c0bb79c97713664a98693e85b1a5bcbcea88b...
15967,POS,FLL,BW,POSFLLBW,c6f18c5cec8758f526d8fca14e205a81236d8122ec7da2...
44239,BEG,ATH,OA,BEGATHOA,d1d1a0c6e7847f0142cd00d79ca0e2d90b0da63773010a...
15993,KIX,PUS,BX,KIXPUSBX,31bb38abf5f5eec009769e1e6210bc965dcad20e10e346...
33886,CGO,HAK,JD,CGOHAKJD,bdbf88317b55a2dd693b13ef358fb7ee8d9d87e89e0d90...
54734,EDI,STN,U2,EDISTNU2,72092f7f7168e3da0d4100f61c99c0f7d634da93b9acbf...


In [63]:
routes_df["hash_key"] = routes_df.apply(lambda x: x["hashed"][0].upper(), axis=1)

In [64]:
routes_df.to_parquet('results/hash/', partition_cols=['hash_key'])

### Part c

In [140]:
import math
import pygeohash as gh

In [141]:
data_centers = {'Location': ['West', 'Central', 'East'],
        'City': ['The Dalles, Oregon', 'Papillion, NE', 'Loudoun County, Virginia'],
        'Latitude': [45.5945645, 41.1544433, 39.08344],
        'Longitude': [-121.1786823, -96.0422378, -77.6497145]}

data_center_df = pd.DataFrame(data_centers)
data_center_df


,Location,City,Latitude,Longitude
0,West,"The Dalles, Oregon",45.594564,-121.178682
1,Central,"Papillion, NE",41.154443,-96.042238
2,East,"Loudoun County, Virginia",39.083440,-77.649715


In [153]:
def get_distance_km(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    km = 6371 * c
    
    return km

def get_closest_data_cetner(lat, long):
    loc_code = [lat,]

    west_dist = get_distance_km((lat, long), (data_center_df.loc[data_center_df['Location'] == "West"]["Latitude"],data_center_df.loc[data_center_df['Location'] == "West"]["Longitude"]))
    cent_dist = get_distance_km((lat, long), (data_center_df.loc[data_center_df['Location'] == "Central"]["Latitude"],data_center_df.loc[data_center_df['Location'] == "Central"]["Longitude"]))
    east_dist = get_distance_km((lat, long), (data_center_df.loc[data_center_df['Location'] == "East"]["Latitude"],data_center_df.loc[data_center_df['Location'] == "East"]["Longitude"]))

    
    if west_dist <= cent_dist and west_dist <= east_dist:
        return "west"
    if cent_dist <= east_dist and cent_dist <= west_dist:
        return "central"
    if east_dist <= west_dist and east_dist <= cent_dist:
        return "east"
    return east_dist

In [155]:
routes_df['location'] = routes_df.apply(lambda x: get_closest_data_cetner(x['src_airport.latitude'], x['src_airport.longitude']), axis=1)
routes_df[['src_airport.iata', 'src_airport.latitude', 'src_airport.longitude', 'key', 'location']].sample(10)

,src_airport.iata,src_airport.latitude,src_airport.longitude,key,location
31656,XIY,34.447102,108.751999,XIYTAOHU,west
51200,UUS,46.888699,142.718002,UUSICNSU,west
44853,SPU,43.538898,16.298000,SPULGWOU,east
16011,TAO,36.266102,120.374001,TAOPUSBX,west
26523,CRL,50.459202,4.453820,CRLPSRFR,east
62019,MCO,28.429399,-81.308998,MCOLAXVX,east
58388,VKO,55.591499,37.261501,VKOMIAUN,east
53901,SIN,1.350190,103.994003,SINPERTR,west
34378,MCP,0.050664,-51.072201,MCPBELJJ,east
1345,DBV,42.561401,18.268200,DBVHAJ4U,east


In [156]:
routes_df.location.value_counts()

east       39278
west       23268
central     4634
Name: location, dtype: int64

In [157]:
routes_df.to_parquet('results/geo/', partition_cols=['location'])

### Part d

In [166]:
def balance_partitions(keys, num_partitions):
    partition_size = (len(keys) + num_partitions - 1) // num_partitions
    partitions = [keys[i:i + partition_size] for i in range(0, len(keys), partition_size)]
    partitions = [sorted(partition) for partition in partitions]
    return partitions

In [170]:
balanced_example = balance_partitions(list(routes_df.key), 30)
print(len(balanced_example))

for partition in balanced_example:
    print(len(partition))

print(balanced_example[0])

30
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2256
2239
['ABJBOY2J', 'ABJOUA2J', 'ACAMEX4O', 'ACATLC4O', 'ACCDKR3G', 'ACCKMS5G', 'ACCLOS3G', 'ACCOUA2J', 'ACCROB3G', 'ACHEBA3L', 'ADQKLN2O', 'ADQnan2O', 'ADQnan2O', 'ADQnan2O', 'ADQnan2O', 'AEPMVD5Q', 'AEPPDP5Q', 'AERKZN2B', 'AGPSTR4U', 'AGPTXL4U', 'AGUMEX4O', 'AJRLYC2N', 'AKInan4Y', 'AKPFAI-', 'AKVYIK-', 'AKVYPX-', 'ALGCGN4U', 'AMDBLR6E', 'AMDBOM6E', 'AMDCCU6E', 'AMDDEL6E', 'AMDGOI6E', 'AMDJAI6E', 'AMDMAA6E', 'AMDPNQ6E', 'AMDnan6E', 'AMSHAM4U', 'AMSNDR3O', 'AMSTNG3O', 'ANFIQQ5P', 'ANUNEV5M', 'ANUPTP3S', 'AOIDUS4U', 'AOITXL4U', 'ARHCSH5N', 'ARHLED5N', 'ARHMMK5N', 'ARHNNM5N', 'ARHSVO5N', 'ARHUSK5N', 'ARNCGN4U', 'ARNDUS4U', 'ARNGEV2N', 'ARNHAD2N', 'ARNHAM4U', 'ARNJKG2N', 'ARNKRF2N', 'ARNKSD2N', 'ARNLYC2N', 'ARNMHQ2N', 'ARNOER2N', 'ARNPOR2N', 'ARNTRF2N', 'ARNVBY2N', 'ARNVHM2N', 'ARNVXO2N', 'ASFDME3R', 'ASFKZN2B', 'ASFLED3R', 'ASFMRV2B', 'ASUMVD5Q

30
